# Property Package Introduction

The properties of the water in WaterTAP models is defined by the [property models](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/index.html). 

In unit models, properties exist as sub-models of the main unit model on what are called _state blocks_. 

The creation of state blocks is done automatically as part of the build for any unit model.

This tutorial will cover the basics of interacting with property packages and state blocks in WaterTAP.

### Required imports from Pyomo, IDAES, and WaterTAP

In [ ]:
from pyomo.environ import ConcreteModel, Constraint, value, units as pyunits

from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.core.solvers import get_solver

### Create model, flowsheet, and property model

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

### Create state block

For the purposes of this tutorial, here we are going to do manually what is done automatically in unit models as part of the build process: create a _state block_.

WaterTAP unit models will typically contain up to three state blocks that define the properties for the inlet, outlet, and waste streams.

In the pump tutorial in week 1, the state block was created on the `control_volume` sub-block as e.g. `m.fs.pump.control_volume.properties_in[0]`. State blocks are always indexed to `0` in all WaterTAP models.

However, despite being termed a "property model", `m.fs.properties` contains no properties on it. Instead, it contains all the information needed to create properties on state blocks.

Using the `build_state_block` method on the property model and passing our index `[0]` will create a standalone state block model with the seawater property model.

In [ ]:
m.fs.state_block = m.fs.properties.build_state_block([0])

Like many other components, we use the `display()` method to see all the variables, constraints, and objectives on the state block.

In this case, we observe only the _state variables_. These are the specific variables used by the property model to define the state of the stream. 

For the seawater property model (any many other WaterTAP property models), the state varaibles are:
- pressure
- temperature
- mass flow rate of components 

For the seawater property model, the components are `H2O` and `TDS`, but other property models allow adding any number of components.

In [ ]:
m.fs.state_block[0].display()

### Creating other properties

Despite the number of other properties available in the seawater property model, we only see the state variables here. Why?

Outside of the state variables, other property variables are only created on state blocks on an as-needed basis. That is, the model will not create these other properties unless they are needed by the model.

To create additional properties, you only need to try to access them (or, "touch" them).

Let's say our model will need the mass concentration of components, named `conc_mass_phase_comp` in our property model. Simply adding a line referencing this (as yet uncreated) model will add this variable to the state block.

In [ ]:
# Touch conc_mass_phase_comp to create it
m.fs.state_block[0].conc_mass_phase_comp
m.fs.state_block[0].display()

Notice that our state block now includes not only `conc_mass_phase_comp`, but also `dens_mass_phase`, `mass_frac_phase_comp`, and `dens_mass_solvent`. Why?

When we created `conc_mass_phase_comp`, we also created the constraints that define `conc_mass_phase_comp`, which is:

`conc_mass_phase_comp = mass_frac_phase_comp * dens_mass_phase`

This likewise created the constraints that define `mass_frac_phase_comp`, and `dens_mass_phase`. 

`mass_frac_phase_comp` is a function of `flow_mass_phase_comp` (our state variable), so that didn't create any additional variables.

But because `dens_mass_phase` is a function of `dens_mass_solvent`, we also see `dens_mass_solvent` on our state block.
    

### Setting the state variables

For all the properties we have created to be constrained, we only need to fix the values for the state variables.

In [ ]:
# fix state variables
m.fs.state_block[0].temperature.fix(273 + 25)  # temperature (K)
m.fs.state_block[0].pressure.fix(101325)  # pressure (Pa)
m.fs.state_block[0].flow_mass_phase_comp["Liq", "H2O"].fix(
    0.965
)  # mass flowrate of H2O (kg/s)
m.fs.state_block[0].flow_mass_phase_comp["Liq", "TDS"].fix(
    0.035
)  # mass flowrate of TDS (kg/s)

print(f"dof = {degrees_of_freedom(m)}")

# display state block
m.fs.state_block[0].display()

### Initialize and solve to determine concentration

With the state variables fixed, we can initialize and solve the model. This will determine the values for all the variables on our state block.

In [ ]:
m.fs.state_block.initialize()

solver = get_solver()
results = solver.solve(m)

### Repeat with osmotic pressure

To further demonstrate this concept, we will repeat the above steps except with the osmotic pressure property `pressure_osm_phase`, which is used in the reverse osmosis model.

In [ ]:
# Create model, flowsheet, and property package
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

# Build state block
m.fs.state_block = m.fs.properties.build_state_block([0])

# Touch the osmotic pressure property
m.fs.state_block[0].pressure_osm_phase
m.fs.state_block[0].conc_mass_phase_comp

# Fix state variables
m.fs.state_block[0].temperature.fix(273 + 25)  # temperature (K)
m.fs.state_block[0].pressure.fix(101325)  # pressure (Pa)
m.fs.state_block[0].flow_mass_phase_comp["Liq", "H2O"].fix(
    0.965
)  # mass flowrate of H2O (kg/s)
m.fs.state_block[0].flow_mass_phase_comp["Liq", "TDS"].fix(
    0.035
)  # mass flowrate of TDS (kg/s)

print(f"dof = {degrees_of_freedom(m)}")

m.fs.state_block.initialize()
results = solver.solve(m)

# Note the additional property variables created on the state block
m.fs.state_block[0].display()

### Convert osmotic pressure units to bar

In [ ]:
# convert osmotic pressure from Pa to bar
pressure_osm_bar = pyunits.convert(
    m.fs.state_block[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
)

# value of the osmotic pressure in bar
print(f"Osmotic pressure: {value(pressure_osm_bar)} bar")

### Solve for other variables using a specified osmotic pressure

Property models can be used to determine the state variables given other variables.

Let's say we know the osmotic pressure of the stream but don't know the mass fractions of water and TDS, but know we have a total mass flow rate of 1 kg/s

In [ ]:
total_mass_flow = 1
m.fs.state_block[0].total_mass_flow_constr = Constraint(
    expr=m.fs.state_block[0].flow_mass_phase_comp["Liq", "H2O"]
    + m.fs.state_block[0].flow_mass_phase_comp["Liq", "TDS"]
    == total_mass_flow
)
# Unfix the previously fixed mass flows
m.fs.state_block[0].flow_mass_phase_comp["Liq", "H2O"].unfix()
m.fs.state_block[0].flow_mass_phase_comp["Liq", "TDS"].unfix()

print(f"dof = {degrees_of_freedom(m)}")

# Fix the osmotic pressure
m.fs.state_block[0].pressure_osm_phase["Liq"].fix(65 * pyunits.bar)

# Solve the state block
results = solver.solve(m)
print(
    f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}"
)

### We see that a osmotic pressure of 65 bar corresponds to a salinity of ~85 g/L TDS

In [ ]:
# Display the state block
m.fs.state_block[0].flow_mass_phase_comp.display()
m.fs.state_block[0].conc_mass_phase_comp.display()

## (Might be a good place to introduce `calculate_state` if we are to do that.)
